In [47]:
import geopandas as gpd
import pandas as pd

In [48]:
ly = gpd.read_file("v10/cd117.gdb/")
congress = pd.read_csv("1976-2020-house.csv")
#select all rows from year 2020
congress = congress[congress['year'] == 2020]

In [57]:
for i, item in congress.iterrows( ):

    val = item["state_po"] + str(item["district"])
    congress.at[i, "code"] = val


for i, item in ly.iterrows():
    val = item["STATE_ABBR"] + item["CDFIPS"]
    ly.at[i, "code"] = val

print(congress.keys())

Index(['year', 'state', 'state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'district', 'stage', 'runoff', 'special', 'candidate',
       'party', 'writein', 'mode', 'candidatevotes', 'totalvotes',
       'unofficial', 'version', 'fusion_ticket', 'code'],
      dtype='object')


In [58]:
grp = congress.groupby("code")

In [59]:
dm = "DEMOCRAT"
rp = "REPUBLICAN"

dists = [["district", "value"]]
for name, group in grp:
    dem = group[group["party"] == dm]
    rep = group[group["party"] == rp]
    votes = dem["candidatevotes"].sum() - rep["candidatevotes"].sum()
    votes = votes / 100
    dists.append([name, votes])

dists = pd.DataFrame(dists[1:], columns=dists[0])

In [60]:
print(dists["value"].min())
print(dists["value"].max())
mx = dists["value"].max()
mn = dists["value"].min()

for i, row in dists.iterrows():
    if row["value"] > 0:
        dists.at[i, "value"] = (row["value"] / mx) * 100
    else:
        dists.at[i, "value"] = (row["value"] / mn) * -100






-3219.84
3629.5


In [74]:

dists.drop(index=0, inplace=True)

In [75]:
for i, item in dists.iterrows():
    dist = item["district"]
    val = item["value"]
    if dist not in ly["code"].values:
        dists.drop(i, inplace=True)




    ly.loc[ly["code"] == dist, "value"] = val





    

AssertionError: District not foundAL1

In [68]:
ly

,DISTRICTID,STFIPS,CDFIPS,STATE_ABBR,NAME,LAST_NAME,PARTY,SQMI,STATE_NAME,Shape_Length,Shape_Area,geometry,code,value
0,0101,01,01,AL,Jerry Carl,Carl,Republican,6223.61,Alabama,22.189776,1.507863,"MULTIPOLYGON (((-88.07826 30.25212, -88.07830 ...",AL01,NaN
1,0102,01,02,AL,Barry Moore,Moore,Republican,10260.03,Alabama,12.165162,2.528306,"MULTIPOLYGON (((-85.05603 32.06306, -85.05593 ...",AL02,NaN
2,0103,01,03,AL,Mike Rogers,Rogers,Republican,7725.80,Alabama,10.369015,1.934011,"MULTIPOLYGON (((-85.47403 34.34455, -85.47045 ...",AL03,NaN
3,0104,01,04,AL,Robert B. Aderholt,Aderholt,Republican,9110.93,Alabama,12.505488,2.305772,"MULTIPOLYGON (((-85.58315 34.86038, -85.58299 ...",AL04,NaN
4,0105,01,05,AL,Mo Brooks,Brooks,Republican,3855.96,Alabama,6.859061,0.983085,"MULTIPOLYGON (((-86.83631 34.99191, -86.83627 ...",AL05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,5505,55,05,WI,Scott Fitzgerald,Fitzgerald,Republican,1955.02,Wisconsin,5.515622,0.560725,"MULTIPOLYGON (((-88.06336 43.19213, -88.06339 ...",WI05,NaN
432,5506,55,06,WI,Glenn Grothman,Grothman,Republican,5210.59,Wisconsin,10.389282,1.509060,"MULTIPOLYGON (((-87.54362 44.32752, -87.54398 ...",WI06,NaN
433,5507,55,07,WI,Thomas P. Tiffany,Tiffany,Republican,23993.41,Wisconsin,25.214416,7.161340,"MULTIPOLYGON (((-92.27452 46.65464, -92.27475 ...",WI07,NaN
434,5508,55,08,WI,Mike Gallagher,Gallagher,Republican,7020.52,Wisconsin,15.798604,2.063791,"MULTIPOLYGON (((-87.98090 44.53818, -87.98061 ...",WI08,NaN
